## 검색 인덱스

In [11]:
# import meilisearch
from meilisearch import Client
import pandas as pd

In [12]:
host = 'http://localhost:7700/'
key = 'aSampleMasterKey'
client = Client(host, key)

- 인덱싱 할 데이터 데이터 프레임으로 로딩

In [13]:
data_file = 'nasdaq_screener_1729815695376.csv'
df = pd.read_csv(data_file, na_filter=False)
df.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$130.69,-2.340,-1.759%,37550853323.00,United States,1999,1830037,Industrials,Biotechnology: Laboratory Analytical Instruments
1,AA,Alcoa Corporation Common Stock,$42.02,0.140,0.334%,10855452683.00,United States,2016,3695426,Industrials,Aluminum
2,AACG,ATA Creativity Global American Depositary Shares,$1.01,0.005,0.498%,32320131.00,China,2008,70350,Real Estate,Other Consumer Services
3,AACT,Ares Acquisition Corporation II Class A Ordina...,$10.86,-0.015,-0.138%,0.00,,2023,65012,Finance,Blank Checks
4,AADI,Aadi Bioscience Inc. Common Stock,$1.95,0.020,1.036%,47998926.00,United States,,54447,Health Care,Biotechnology: Pharmaceutical Preparations


In [14]:
df['id'] = df['Symbol'].str.strip().replace(r'[/^]', '_', regex=True)
df

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,id
0,A,Agilent Technologies Inc. Common Stock,$130.69,-2.340,-1.759%,37550853323.00,United States,1999,1830037,Industrials,Biotechnology: Laboratory Analytical Instruments,A
1,AA,Alcoa Corporation Common Stock,$42.02,0.140,0.334%,10855452683.00,United States,2016,3695426,Industrials,Aluminum,AA
2,AACG,ATA Creativity Global American Depositary Shares,$1.01,0.005,0.498%,32320131.00,China,2008,70350,Real Estate,Other Consumer Services,AACG
3,AACT,Ares Acquisition Corporation II Class A Ordina...,$10.86,-0.015,-0.138%,0.00,,2023,65012,Finance,Blank Checks,AACT
4,AADI,Aadi Bioscience Inc. Common Stock,$1.95,0.020,1.036%,47998926.00,United States,,54447,Health Care,Biotechnology: Pharmaceutical Preparations,AADI
...,...,...,...,...,...,...,...,...,...,...,...,...
7021,ZVRA,Zevra Therapeutics Inc. Common Stock,$7.90,-0.110,-1.373%,415680533.00,United States,,390995,Health Care,Biotechnology: Pharmaceutical Preparations,ZVRA
7022,ZVSA,ZyVersa Therapeutics Inc. Common Stock,$2.07,-0.070,-3.271%,2130436.00,United States,2022,101857,Health Care,Biotechnology: Pharmaceutical Preparations,ZVSA
7023,ZWS,Zurn Elkay Water Solutions Corporation Common ...,$36.30,0.170,0.471%,6204704369.00,United States,2012,936476,Industrials,Industrial Machinery/Components,ZWS
7024,ZYME,Zymeworks Inc. Common Stock,$12.92,-0.190,-1.449%,917932279.00,United States,,247917,Health Care,Biotechnology: Pharmaceutical Preparations,ZYME


- index 생성, 서버에 추가

In [15]:
# nasdaq라는 이름으로 index 생성, dict로 변경을 해야함. 
# -> indexing 결과를 meilisearch 서치 엔진에 추가함
client.index('nasdaq').add_documents(df.to_dict(orient='records'), primary_key='id')


TaskInfo(task_uid=1, index_uid='nasdaq', status='enqueued', type='documentAdditionOrUpdate', enqueued_at=datetime.datetime(2024, 10, 25, 1, 47, 23, 900395))

- 검색 테스트

In [16]:
# word = "Apple"
# word = "Apple Inc."
word = "AAPL"
client.index('nasdaq').search(word)

{'hits': [{'id': 'AAPL',
   'Symbol': 'AAPL',
   'Name': 'Apple Inc. Common Stock',
   'Last Sale': '$230.57',
   'Net Change': -0.19,
   '% Change': '-0.082%',
   'Market Cap': '3505617868090.00',
   'Country': 'United States',
   'IPO Year': '1980',
   'Volume': 31051206,
   'Sector': 'Technology',
   'Industry': 'Computer Manufacturing'}],
 'query': 'AAPL',
 'processingTimeMs': 0,
 'limit': 20,
 'offset': 0,
 'estimatedTotalHits': 1}